In [22]:
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.utils import resample


In [10]:
df = pd.read_csv('data/labeled_data.csv')
df.dropna(inplace=True)
df.drop_duplicates(['DATE'], inplace=True)

In [11]:
df

,BID_PRICE,CONTENT,DATE,PRICE,TIME,TITLE,annotation_id,annotator,created_at,id,index,lead_time,sentiment,updated_at
0,48.65,Mở màn phiên giao dịch ngày đầu tiên của tháng...,01/06/2020,49.00,09:30,Giá vàng ngày 1/6/2020: đồng loạt tăng lên mốc...,7,1,2024-05-30T01:57:35.410889Z,782,0,18.027,Sell,2024-05-30T01:57:45.330205Z
1,48.68,"Trái ngược với đà tăng ngày hôm qua, hôm nay 2...",02/06/2020,49.03,09:10,Giá vàng ngày 2/6/2020: đảo chiều giảm,8,1,2024-05-30T01:58:13.990422Z,783,1,12.532,Buy,2024-05-30T01:58:13.990422Z
2,48.52,Giá vàng ngày 3/6/2020 tiếp tục giảm so với hô...,03/06/2020,48.87,11:00,Giá vàng ngày 3/6/2020: tiếp đà giảm giá,9,1,2024-05-30T02:28:08.312632Z,784,2,1793.447,Buy,2024-05-30T02:28:08.312632Z
3,48.38,Giá vàng ngày 4/6/2020 tiếp tục ghi nhận sự sụ...,04/06/2020,48.73,08:25,Giá vàng ngày 4/6/2020: trong nước và thế giới...,10,1,2024-05-30T02:28:13.270395Z,785,3,4.625,Buy,2024-05-30T02:28:13.270395Z
4,48.50,"Chấm dứt chuỗi ngày giảm liên tiếp, giá vàng h...",05/06/2020,48.84,10:15,Giá vàng ngày 5/6/2020: trong nước và thế giới...,11,1,2024-05-30T02:28:16.563066Z,786,4,3.000,Sell,2024-05-30T02:28:16.563066Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,85.70,"Bất chấp vàng thế giới giảm, vàng SJC vẫn tăng...",09/05/2024,88.00,08:35,"Giá vàng hôm nay 9/5: Tăng dựng đứng, vàng SJC...",781,1,2024-05-30T05:38:53.048993Z,1556,774,0.949,Sell,2024-05-30T05:38:53.048993Z
775,87.20,Vàng thế giới và trong nước đồng loạt kéo nhau...,10/05/2024,89.50,08:35,"Giá vàng hôm nay 10/5: Điên cuồng tăng giá, và...",782,1,2024-05-30T05:38:55.035292Z,1557,775,1.331,Sell,2024-05-30T05:38:55.035292Z
776,88.80,"Tính từ đầu tuần đến nay, giá vàng SJC đã tăng...",11/05/2024,91.30,09:50,Giá vàng hôm nay 11/5: Vàng SJC đã tăng tới 6 ...,783,1,2024-05-30T05:38:58.817382Z,1558,776,3.016,Sell,2024-05-30T05:38:58.818343Z
777,85.50,"Mở cửa phiên giao dịch đầu tuần, giá vàng SJC ...",13/05/2024,88.50,09:00,"Giá vàng hôm nay 13/5: Vàng SJC ""bay"" hơn 3 tr...",784,1,2024-05-30T05:39:01.501133Z,1559,777,2.016,Sell,2024-05-30T05:39:01.501133Z


In [12]:
df.drop(['annotation_id', 	'annotator',	'created_at', 	'id',	'index',	'lead_time', 'updated_at'], axis=1, inplace=True)

In [13]:
df.rename(columns={'sentiment': 'labels'}, inplace=True)

In [21]:
df

,BID_PRICE,CONTENT,DATE,PRICE,TIME,TITLE,labels
0,48.65,Mở màn phiên giao dịch ngày đầu tiên của tháng...,01/06/2020,49.00,09:30,Giá vàng ngày 1/6/2020: đồng loạt tăng lên mốc...,increase
1,48.68,"Trái ngược với đà tăng ngày hôm qua, hôm nay 2...",02/06/2020,49.03,09:10,Giá vàng ngày 2/6/2020: đảo chiều giảm,decrease
2,48.52,Giá vàng ngày 3/6/2020 tiếp tục giảm so với hô...,03/06/2020,48.87,11:00,Giá vàng ngày 3/6/2020: tiếp đà giảm giá,decrease
3,48.38,Giá vàng ngày 4/6/2020 tiếp tục ghi nhận sự sụ...,04/06/2020,48.73,08:25,Giá vàng ngày 4/6/2020: trong nước và thế giới...,decrease
4,48.50,"Chấm dứt chuỗi ngày giảm liên tiếp, giá vàng h...",05/06/2020,48.84,10:15,Giá vàng ngày 5/6/2020: trong nước và thế giới...,increase
...,...,...,...,...,...,...,...
774,85.70,"Bất chấp vàng thế giới giảm, vàng SJC vẫn tăng...",09/05/2024,88.00,08:35,"Giá vàng hôm nay 9/5: Tăng dựng đứng, vàng SJC...",increase
775,87.20,Vàng thế giới và trong nước đồng loạt kéo nhau...,10/05/2024,89.50,08:35,"Giá vàng hôm nay 10/5: Điên cuồng tăng giá, và...",increase
776,88.80,"Tính từ đầu tuần đến nay, giá vàng SJC đã tăng...",11/05/2024,91.30,09:50,Giá vàng hôm nay 11/5: Vàng SJC đã tăng tới 6 ...,increase
777,85.50,"Mở cửa phiên giao dịch đầu tuần, giá vàng SJC ...",13/05/2024,88.50,09:00,"Giá vàng hôm nay 13/5: Vàng SJC ""bay"" hơn 3 tr...",increase


Change labels from buy sell to incrase, decrease

In [15]:
def change_labels(label):
    if label == 'Sell':
        label = 'increase'
    elif label == 'Buy': 
        label = 'decrease'
    return label

In [16]:
df['labels'] = df['labels'].apply(lambda x: change_labels(x))

In [17]:
Counter(df['labels'])

Counter({'decrease': 411, 'increase': 368})

In [19]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [20]:
train.to_csv('./data/train_data.csv')
test.to_csv('./data/test_data.csv')

### Balance data

In [23]:
# Tách các bài báo thành các nhóm 'increase' và 'decrease'
increase_data = df[df['labels'] == 'increase']
decrease_data = df[df['labels'] == 'decrease']

In [24]:
# Kiểm tra số lượng mẫu trong từng nhãn
print(f"Original 'increase' samples: {len(increase_data)}")
print(f"Original 'decrease' samples: {len(decrease_data)}")


Original 'increase' samples: 368
Original 'decrease' samples: 411


In [25]:
# Random undersample nhãn nhiều hơn để có số lượng mẫu cân bằng
if len(increase_data) > len(decrease_data):
    increase_data_downsampled = resample(increase_data,
                                         replace=False,
                                         n_samples=len(decrease_data),
                                         random_state=42)
    balanced_df = pd.concat([increase_data_downsampled, decrease_data])
else:
    decrease_data_downsampled = resample(decrease_data,
                                         replace=False,
                                         n_samples=len(increase_data),
                                         random_state=42)
    balanced_df = pd.concat([increase_data, decrease_data_downsampled])

# Shuffle the dataframe to mix the samples
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [26]:
# Kiểm tra lại số lượng mẫu sau khi downsample
print(f"Balanced 'increase' samples: {len(balanced_df[balanced_df['labels'] == 'increase'])}")
print(f"Balanced 'decrease' samples: {len(balanced_df[balanced_df['labels'] == 'decrease'])}")

Balanced 'increase' samples: 368
Balanced 'decrease' samples: 368


In [27]:
balanced_df

,BID_PRICE,CONTENT,DATE,PRICE,TIME,TITLE,labels
0,55.70,Giá vàng hôm nay giảm nhẹ từ 50.000 đến 150.00...,08/09/2020,56.50,08:35,Giá vàng hôm nay ngày 8/9/2020: quay đầu giảm,decrease
1,56.75,Giá vàng lúc 9h00 ngày 4/8/2020 tăng lên mốc 4...,04/08/2020,57.90,08:35,Giá vàng ngày 4/8/2020: tăng lên mốc 58 triệu,increase
2,66.20,Tài khoản không đủ để thực hiện chức năng này....,20/08/2022,67.20,08:35,Giá vàng hôm nay 20/8: Tiếp tục giảm sâu chưa ...,decrease
3,65.20,. Giá vàng thế giới và trong nước đều bật tăng...,04/10/2022,66.20,08:15,"Giá vàng hôm nay 4/10: Tăng phi mã, chấm dứt c...",increase
4,68.00,Giá kim loại quý đạt được mức tăng khiêm tốn v...,16/09/2023,68.70,08:30,Giá vàng hôm nay 16/9: Quay đầu tăng nhẹ phiên...,increase
...,...,...,...,...,...,...,...
731,55.05,Giá vàng hôm nay 8/12: Sau nhiều phiên giao dị...,08/12/2020,55.60,11:05,"Giá vàng hôm nay ngày 08/12: Đảo chiều, vọt tă...",increase
732,54.97,Giá vàng hôm nay ngày 14/4/2021 tăng từ 150.00...,14/04/2021,55.37,08:40,Giá vàng hôm nay ngày 14/4/2021: vàng tăng mạn...,increase
733,69.30,"Ghi nhận sáng nay, giá vàng SJC bán ra đã chạm...",12/10/2023,70.00,08:35,Giá vàng hôm nay 12/10: Vàng SJC phăm phăm tiế...,increase
734,66.15,". Giá vàng vừa trải qua một tuần giảm sâu, áp ...",10/09/2022,66.95,08:35,Giá vàng hôm nay 10/9: Giảm thê thảm tuần thứ ...,decrease


In [30]:
train_balance, test_balance = train_test_split(balanced_df, test_size=0.2, random_state=42)

In [31]:
train_balance.to_csv('./data/balance_train.csv', index=False)
test_balance.to_csv('./data/balance_test.csv', index=False)